# Explore here

In [25]:
# Your code here
from sqlalchemy import create_engine, text
import pandas as pd
from dotenv import load_dotenv
import os
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import pickle


In [26]:
load_dotenv()

True

In [27]:
try:
    connection_string = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
    engine = create_engine(connection_string).execution_options(autocommit=True)
    connection = engine.connect()
    print("[INFO] Connected to database")
except Exception as e:
    print(f"[ERROR] Error connecting to database: {e}")

[INFO] Connected to database


In [28]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
        CREATE TABLE IF NOT EXISTS movies (
            id INT PRIMARY KEY,
            budget INT,
            genres TEXT,
            homepage TEXT,
            keywords TEXT,
            original_language TEXT,
            original_title TEXT,
            overview TEXT,
            popularity FLOAT,
            production_companies TEXT,
            production_countries TEXT,
            release_date TEXT,
            revenue INT,
            runtime FLOAT,
            spoken_languages TEXT,
            txt_status TEXT,
            tagline TEXT,
            title TEXT,
            vote_average FLOAT,
            vote_count INT
        );
        """))
        print("[INFO] movies table created successfully")
except Exception as e:
    print(f"[ERROR] Error creating movies table: {e}")

[INFO] movies table created successfully


In [29]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
        CREATE TABLE IF NOT EXISTS credits (
            movie_id INT,
            title TEXT,
            txt_cast TEXT,
            crew TEXT
        );
        """))
        print("[INFO] credits table created successfully")
except Exception as e:
    print(f"[ERROR] Error creating credits table: {e}")

[INFO] credits table created successfully


In [30]:
try:
    df_movies = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv', delimiter=',')
except Exception as e:
    print(f"[INFO] Error reading the file from the URL: {e}")

try:
    df_credits = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv', delimiter=',')
except Exception as e:
    print(f"[ERROR] Error reading the file from the URL: {e}")

In [31]:
df_movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [32]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [33]:
df_credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [34]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [35]:
try:
    df_movies.to_sql('movies', engine, if_exists='replace', index=False)
    print("[INFO] Data written to the database in the movies table")
except Exception as e:
    print(f"[ERROR] Error writing to the database: {e}")

try:
    df_credits.to_sql('credits', engine, if_exists='replace', index=False)
    print("[INFO] Data written to the database in the credits table")
except Exception as e:
    print(f"[ERROR] Error writing to the database: {e}")

[INFO] Data written to the database in the movies table
[INFO] Data written to the database in the credits table


In [36]:
query = """
    SELECT *
    FROM movies
    INNER JOIN credits
    ON movies.title = credits.title;
"""
try:
    with engine.connect() as connection:
        movies_table = pd.read_sql_query(query, engine)
        print("[INFO] Data read from the database")
        connection.close()
except Exception as e:
    print(f"[ERROR] Error reading from the database: {e}")


[INFO] Data read from the database


In [37]:
movies_table = movies_table.loc[:, ~movies_table.columns.duplicated()]
movies_table = movies_table[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies_table.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,10481,102 Dalmatians,Get ready for a howling good time as an all ne...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...","[{""id"": 212, ""name"": ""london england""}, {""id"":...","[{""cast_id"": 1, ""character"": ""Cruella de Vil"",...","[{""credit_id"": ""5539fe5e9251413f5a00359b"", ""de..."
1,333371,10 Cloverfield Lane,"After a car accident, Michelle awakens to find...","[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...","[{""id"": 1930, ""name"": ""kidnapping""}, {""id"": 23...","[{""cast_id"": 2, ""character"": ""Michelle"", ""cred...","[{""credit_id"": ""57627624c3a3680682000872"", ""de..."
2,345003,10 Days in a Madhouse,"Nellie Bly, a 23 year-old reporter for Joseph ...","[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 1568, ""name"": ""undercover""}, {""id"": 49...","[{""cast_id"": 2, ""character"": ""Nellie Bly"", ""cr...","[{""credit_id"": ""594efa1fc3a36832650455ff"", ""de..."
3,4951,10 Things I Hate About You,"Bianca, a tenth grader, has never gone on a da...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...","[{""id"": 497, ""name"": ""shakespeare""}, {""id"": 59...","[{""cast_id"": 2, ""character"": ""Patrick Verona"",...","[{""credit_id"": ""52fe43e6c3a36847f807731d"", ""de..."
4,13197,10th & Wolf,A former street tough returns to his Philadelp...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 1568, ""name"": ""undercover""}, {""id"": 10...","[{""cast_id"": 1, ""character"": ""Tommy"", ""credit_...","[{""credit_id"": ""52fe454d9251416c75051e59"", ""de..."


In [38]:
def get_json(json_str, default_value=None):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return default_value

def extract_names_from_json(json_list):
    if isinstance(json_list, list):
        return [item['name'] for item in json_list if isinstance(item, dict) and 'name' in item]
    return []

def extract_cast_from_json(json_list):
    if isinstance(json_list, list):
        return [item['name'] for item in json_list[:3] if isinstance(item, dict) and 'name' in item]
    return []

def extract_director_from_json(json_list):
    if isinstance(json_list, list):
        for item in json_list:
            if isinstance(item, dict) and item.get('job') == 'Director' and 'name' in item:
                return item['name']
    return ""

def process_row(row):
    genres = extract_names_from_json(get_json(row['genres']))
    keywords = extract_names_from_json(get_json(row['keywords']))
    cast = extract_cast_from_json(get_json(row['cast']))
    director = extract_director_from_json(get_json(row['crew']))

    overview = [str(row['overview'])] if pd.notna(row['overview']) else []

    tags = overview + genres + keywords + cast + [director]
    return " ".join(filter(None, (" ".join(tag.split()) for tag in map(str, tags))))


In [39]:
movies_table['tags'] = movies_table.apply(process_row, axis=1)

columns_to_drop = ['genres', 'keywords', 'cast', 'crew', 'overview']
movies_table.drop(columns=columns_to_drop, inplace=True)

movies_table["tags"][0]


"Get ready for a howling good time as an all new assortment of irresistible animal heroes are unleashed in this great family tail! In an unlikely alliance, the outrageous Waddlesworth... a parrot who thinks he's a Rottweiler... teams up with Oddball... an un-marked Dalmation puppy eager to earn her spots! Together they embark on a laugh-packed quest to outwit the ever-scheming Cruella De Vil Comedy Family london england prison release from prison women's prison society for the prevention of cruelty to animals puppy pelz dog dalmatian Glenn Close Ioan Gruffudd Alice Evans Kevin Lima"

In [40]:
movies_table.to_csv("../data/processed/movies_tags.csv", index=False)

movies_table.head(5)


,movie_id,title,tags
0,10481,102 Dalmatians,Get ready for a howling good time as an all ne...
1,333371,10 Cloverfield Lane,"After a car accident, Michelle awakens to find..."
2,345003,10 Days in a Madhouse,"Nellie Bly, a 23 year-old reporter for Joseph ..."
3,4951,10 Things I Hate About You,"Bianca, a tenth grader, has never gone on a da..."
4,13197,10th & Wolf,A former street tough returns to his Philadelp...


In [41]:
try:
    movies_table.to_sql('movies_tags', engine, if_exists='replace', index=False)
    print("[INFO] Data written to the database in the movies_tags table")
except Exception as e:
    print(f"[ERROR] Error writing to the database: {e}")

[INFO] Data written to the database in the movies_tags table


In [42]:
tfidfVectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix= tfidfVectorizer.fit_transform(movies_table['tags'])
similarity = cosine_similarity(tfidf_matrix)

In [43]:
def recommend(movie):
    """ Function to recommend movies based on the input movie """
    movie_list = sorted(list(enumerate(similarity[movies_table[movies_table["title"] == movie].index[0]])), reverse = True , key = lambda x: x[1])[1:6]
    for i in movie_list:
        print(movies_table.iloc[i[0]].title)

In [44]:
recommend("Aliens")

Alien³
Avatar
Alien
Alien: Resurrection
Mission to Mars


In [45]:
knn = NearestNeighbors(n_neighbors=6, metric='cosine')
knn.fit(tfidf_matrix)


NearestNeighbors(metric='cosine', n_neighbors=6)

In [46]:
def recommend_knn(movie_title):
    """Function to recommend movies using KNN."""
    try:
        movie_index = movies_table[movies_table["title"] == movie_title].index[0]
        _, indices = knn.kneighbors(tfidf_matrix[movie_index])

        print(f"Movies similar to'{movie_title}':")
        for i in range(1, len(indices[0])):
            print(movies_table.iloc[indices[0][i]].title)
    except IndexError:
        print(f"Movie '{movie_title}' Not found")

In [47]:
recommend_knn("Aliens")

Movies similar to'Aliens':
Alien³
Avatar
Alien
Alien: Resurrection
Mission to Mars


In [49]:
pickle.dump(knn, open("../models/knn_neighbors-6_algorithm-brute_metric-cosine.sav", "wb"))

In [ ]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
        DROP TABLE IF EXISTS movies; """))
        connection.execute(text("""
        DROP TABLE IF EXISTS credits;"""))
        connection.execute(text("""
        DROP TABLE IF EXISTS movies_tags;
        """))
        connection.close()
    print("[INFO] Tables dropped successfully")
except Exception as e:
    print(f"[ERROR] Error dropping tables: {e}")


[INFO] Tables dropped successfully
